# **UNIVERSIDAD TORCUATO DI TELLA**
## **MAESTRÍA EN ECONOMETRÍA**

---

### **TRABAJO PRÁCTICO DE ECONOMETRÍA**

- **Profesor:** González-Rozada, Martín  
- **Ayudante:** Lening, Iara  
- **Alumno:** Guzzi, David Alexander  (Legajo n°: 24H1970)  

**Ciclo Lectivo:** Tercer Trimestre, 2024  

---

##### **1. PROPIEDADES DE MUESTRA FINITA DE FGLS (MCGE).**

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Parámetros del modelo
beta_0 = -3
beta_1 = 0.8
omega = np.diag([4, 9, 16, 25, 36])  # Matriz de varianzas y covarianzas de u
n_observations = 5  # Tamaño de cada muestra
n_samples = 5000  # Número de muestras


# Generación de muestras
samples = []
np.random.seed(3649)  #Últimos 4 números de mi documento de identidad.

for _ in range(n_samples):
    x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
    u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
    y = beta_0 + beta_1 * x + u  # Generar y
    samples.append(pd.DataFrame({'x': x, 'u': u, 'y': y}))

# Combinar todas las muestras en un único DataFrame
all_samples = pd.concat(samples, keys=range(n_samples), names=['sample', 'index']).reset_index()

# Mostrar algunas filas
all_samples.head(10)

,sample,index,x,u,y
0,0,0,44.691948,0.394367,33.147926
1,0,1,33.945501,-2.739349,21.417052
2,0,2,37.364044,-5.117829,21.773406
3,0,3,37.251050,1.502358,28.303198
4,0,4,34.525174,6.038023,30.658162
5,1,0,3.409759,0.030947,-0.241246
6,1,1,14.352028,-0.504023,7.977600
7,1,2,30.112575,2.001504,23.091564
8,1,3,5.797610,3.095406,4.733495
9,1,4,20.579967,8.232626,21.696599


In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Configuración inicial
np.random.seed(3649)

# Parámetros iniciales
beta_0_true = -3
beta_1_true = 0.8
omega = np.diag([4, 9, 16, 25, 36])  # Matriz de varianzas
n_samples = 5000
n_observations = 5

x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
x2 = x**2
u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
y = beta_0_true + beta_1_true * x + u  # Generar y

# Creamos un DataFrame
df = pd.DataFrame({
    'x': x,
    'x2': x2,
    'u': u,
    'y': y
})

# 1. Estimar modelo inicial y ~ x por OLS y obtener las estimaciones de los parámetros del modelo
X = sm.add_constant(x)  # Matriz de diseño con intercepto
ols_model = sm.OLS(y, X)  # Estimación OLS
ols_results = ols_model.fit()  # Ajuste de OLS

# 2. Calcular los residuos del modelo y elevarlos al cuadrado
df['u_hat'] = ols_results.resid
df['u_hat2'] = df['u_hat']**2

# 3. Dada la forma funcional de la heterocedasticidad de White para este modelo, sigma2 ~ x x2, se estima por OLS usando u_hat2 como proxy de sigma2
ols_model_aux = sm.OLS(df['u_hat2'], df[['x', 'x2']]) #Ver de agregar intercepto.
ols_model_aux_results = ols_model_aux.fit()
gamma_hat1 = ols_model_aux_results.params['x']
gamma_hat2 = ols_model_aux_results.params['x2']

# 4. Usar las estimaciones de la regresión auxiliar y obtener las varianzas ajustadas (no consistentes)
df['sigma2_hat'] = ols_model_aux_results.predict(df[['x', 'x2']])

# 5. Transformar las variables de la forma funcional de la heterocedasticidad de White dividiéndolas por la raíz de sigma2_hat y estimar por OLS
df['u_hat2_over_sigma2_hat'] = df['u_hat2'] / df['sigma2_hat']
df['x_over_sigma2_hat'] = df['x'] / df['sigma2_hat']
df['x2_over_sigma2_hat'] = df['x2'] / df['sigma2_hat']

ols_model_aux2 = sm.OLS(df['u_hat2_over_sigma2_hat'], df[['x_over_sigma2_hat', 'x2_over_sigma2_hat']]) #Ver de agregar intercepto.
ols_model_aux_results2 = ols_model_aux2.fit()
gamma_tilde1 = ols_model_aux_results2.params['x_over_sigma2_hat']
gamma_tilde2 = ols_model_aux_results2.params['x2_over_sigma2_hat']

# 6. Usar las estimaciones de la regresión auxiliar 2 y obtener las varianzas ajustadas (consistentes)
df['sigma2_tilde'] = ols_model_aux_results2.predict(df[['x_over_sigma2_hat', 'x2_over_sigma2_hat']])
df['sigma2_tilde'] = np.sqrt(df['sigma2_tilde'])

# 7. Usar uno sobre sigma tilde como ponderador en la regresión de y sobre x
df['y_estrella'] = df['y'] / df['sigma_tilde']
df['x_estrella'] = df['x'] / df['sigma_tilde']

X_estrella = sm.add_constant(df['x_over_sigma_tilde']) 
final_ols_model = sm.OLS(df['y_estrella'], X_estrella)
final_ols_results = final_ols_model.fit() 

gamma_hat: 1.0190342558308039
beta1 (coeficiente de x): 1.049125141002301
beta0 (intercepto): 1.1894337904253731


In [11]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Configuración inicial
np.random.seed(3649)

# Parámetros iniciales
beta_0_true = -3
beta_1_true = 0.8
omega = np.diag([4, 9, 16, 25, 36])  # Matriz de varianzas
#n_samples = 5000
n_observations = 5

x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
x2 = x**2
u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
y = beta_0_true + beta_1_true * x + u  # Generar y

# Creamos un DataFrame
df = pd.DataFrame({
    'x': x,
    'x2': x2,
    'u': u,
    'y': y
})

# 1. Estimar modelo inicial y ~ x por OLS y obtener las estimaciones de los parámetros del modelo
X = sm.add_constant(x)  # Matriz de diseño con intercepto
ols_model = sm.OLS(y, X)  # Estimación OLS
ols_results = ols_model.fit()  # Ajuste de OLS

# 2. Calcular los residuos del modelo y elevarlos al cuadrado
df['u_hat'] = ols_results.resid
df['u_hat2'] = df['u_hat']**2

# 3. Regresión auxiliar: u_hat2 ~ x + x2 (incluyendo intercepto)
aux_X = sm.add_constant(df[['x', 'x2']])
ols_model_aux = sm.OLS(df['u_hat2'], aux_X)
ols_model_aux_results = ols_model_aux.fit()
gamma_hat1 = ols_model_aux_results.params['x']
gamma_hat2 = ols_model_aux_results.params['x2']

# 4. Usar las estimaciones para obtener las varianzas ajustadas
df['sigma2_hat'] = ols_model_aux_results.predict(aux_X)

# 5. Transformar las variables de la forma funcional de la heterocedasticidad de White dividiéndolas por la raíz de sigma2_hat y estimar por OLS
df['u_hat2_over_sigma2_hat'] = df['u_hat2'] / df['sigma2_hat']
df['x_over_sigma2_hat'] = df['x'] / df['sigma2_hat']
df['x2_over_sigma2_hat'] = df['x2'] / df['sigma2_hat']

aux_X_2 = sm.add_constant(df[['x_over_sigma2_hat', 'x2_over_sigma2_hat']])
ols_model_aux2 = sm.OLS(df['u_hat2_over_sigma2_hat'], aux_X_2)
ols_model_aux_results2 = ols_model_aux2.fit()
gamma_tilde1 = ols_model_aux_results2.params['x_over_sigma2_hat']
gamma_tilde2 = ols_model_aux_results2.params['x2_over_sigma2_hat']

# 6. Usar las estimaciones de la regresión auxiliar 2 y obtener las varianzas ajustadas (consistentes)
df['sigma2_tilde'] = ols_model_aux_results2.predict(aux_X_2)
df['sigma_tilde'] = np.sqrt(df['sigma2_tilde'])

# 7. Usar uno sobre sigma tilde como ponderador en la regresión de y sobre x
df['y_star'] = df['y'] / df['sigma_tilde']
df['x_star'] = df['x'] / df['sigma_tilde']

aux_X_star = sm.add_constant(df['x_star']) 
final_ols_model = sm.OLS(df['y_star'], aux_X_star)
final_ols_results = final_ols_model.fit() 

# Estimaciones finales
beta_0_hat = final_ols_results.params['const']
beta_1_hat = final_ols_results.params['x_star']
print(f"Estimated beta_0: {beta_0_hat:.4f}, beta_1: {beta_1_hat:.4f}")

Estimated beta_0: 19.1022, beta_1: 0.1914


In [17]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from scipy.stats import t

# Configuración inicial
np.random.seed(3649)

# Parámetros iniciales
beta_0_true = -3
beta_1_true = 0.8
omega = np.diag([4, 9, 16, 25, 36])  # Matriz de varianzas
n_samples = 1
n_observations = 5

# Función para realizar FGLS
def fgls_estimation(x, y):
    # DataFrame para almacenar las variables intermedias
    df = pd.DataFrame({'x': x, 'y': y})
    df['x2'] = df['x'] ** 2

    # 1. Estimar modelo inicial y ~ x por OLS y obtener las estimaciones de los parámetros del modelo
    X = sm.add_constant(x)  # Matriz de diseño con intercepto
    ols_model = sm.OLS(y, X)  # Estimación OLS
    ols_results = ols_model.fit()  # Ajuste de OLS

    # 2. Calcular los residuos del modelo y elevarlos al cuadrado
    df['u_hat'] = ols_results.resid
    df['u_hat2'] = df['u_hat']**2

    # 3. Dada la forma funcional de la heterocedasticidad de White para este modelo, sigma2 ~ x + x2, se estima por OLS usando u_hat2 como proxy de sigma2
    aux_X = sm.add_constant(df[['x', 'x2']])
    ols_model_aux = sm.OLS(df['u_hat2'], aux_X)
    ols_model_aux_results = ols_model_aux.fit()
    gamma_hat1 = ols_model_aux_results.params['x']
    gamma_hat2 = ols_model_aux_results.params['x2']

    # 4. Usar las estimaciones de la regresión auxiliar y obtener las varianzas ajustadas (no consistentes)
    df['sigma2_hat'] = ols_model_aux_results.predict(aux_X)

    # 5. Transformar las variables de la forma funcional de la heterocedasticidad de White dividiéndolas por la raíz de sigma2_hat y estimar por OLS
    df['u_hat2_over_sigma2_hat'] = df['u_hat2'] / df['sigma2_hat']
    df['x_over_sigma2_hat'] = df['x'] / df['sigma2_hat']
    df['x2_over_sigma2_hat'] = df['x2'] / df['sigma2_hat']

    aux_X_2 = sm.add_constant(df[['x_over_sigma2_hat', 'x2_over_sigma2_hat']])
    ols_model_aux2 = sm.OLS(df['u_hat2_over_sigma2_hat'], aux_X_2)
    ols_model_aux_results2 = ols_model_aux2.fit()
    gamma_tilde1 = ols_model_aux_results2.params['x_over_sigma2_hat']
    gamma_tilde2 = ols_model_aux_results2.params['x2_over_sigma2_hat']

    # 6. Usar las estimaciones de la regresión auxiliar 2 y obtener las varianzas ajustadas (consistentes)
    df['sigma2_tilde'] = ols_model_aux_results2.predict(aux_X_2)
    
    # Corregir valores negativos o muy pequeños
    df['sigma2_tilde'] = df['sigma2_tilde'].clip(lower=1e-10)
    df['sigma_tilde'] = np.sqrt(df['sigma2_tilde'])

    # Verificar si hay valores NaN
    if df['sigma_tilde'].isna().any():
        raise ValueError("Se encontraron valores NaN en sigma_tilde")

    # 7. Usar uno sobre sigma tilde como ponderador en la regresión de y ~ x
    df['y_star'] = df['y'] / df['sigma_tilde']
    df['x_star'] = df['x'] / df['sigma_tilde']

    aux_X_star = sm.add_constant(df['x_star']) 
    final_ols_model = sm.OLS(df['y_star'], aux_X_star)
    final_ols_results = final_ols_model.fit() 

    # Estimaciones finales
    beta_0_hat = final_ols_results.params['const']
    beta_1_hat = final_ols_results.params['x_star']
    se_beta1_hat = final_ols_results.bse.iloc[1]

    return beta_0_hat, beta_1_hat, se_beta1_hat

# Inicializar listas para almacenar resultados
samples = []
results = []

# Generar muestras y aplicar FGLS
for i in range(n_samples):
    # Generar datos
    x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
    u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
    y = beta_0_true + beta_1_true * x + u  # Generar y

    # Almacenar muestra
    samples.append(pd.DataFrame({'sample': i, 'x': x, 'y': y, 'u': u}))

    # Aplicar FGLS
    beta_0_hat, beta_1_hat, se_beta1_hat = fgls_estimation(x, y)
    # t_stat = (beta_1_hat - beta_1_true) / se_beta[1]  # Estadístico t para beta_1
    # p_value = 2 * (1 - t.cdf(abs(t_stat), df=n_observations - 2))  # Valor p

    # Almacenar resultados
    results.append([i, beta_0_hat, beta_1_hat, se_beta1_hat])

# Combinar todas las muestras en un único DataFrame
all_samples = pd.concat(samples).reset_index(drop=True)

# Crear DataFrame con los resultados de FGLS
results_df = pd.DataFrame(results, columns=[
    'sample', 'beta_0_hat', 'beta_1_hat', 'se_beta1_hat'
])

# Mostrar ejemplos de los DataFrames generados
print("Ejemplo de all_samples:")
print(all_samples.head())
print("\nEjemplo de results_df:")
print(results_df.head())

Ejemplo de all_samples:
   sample          x          y         u
0       0  44.691948  33.147926  0.394367
1       0  33.945501  21.417052 -2.739349
2       0  37.364044  21.773406 -5.117829
3       0  37.251050  28.303198  1.502358
4       0  34.525174  30.658162  6.038023

Ejemplo de results_df:
   sample  beta_0_hat  beta_1_hat  se_beta1_hat
0       0   19.102219    0.191446      1.192612


In [15]:
results_df.head(1)

,sample,beta_0_hat,beta_1_hat,se_beta1_hat
0,0,19.102219,0.191446,1.192612


In [9]:
results_df['beta_1_hat'].median()

0.7429979282613803

In [11]:
all_samples.shape

(25000, 4)

In [10]:
results_df.shape

(5000, 3)

In [5]:
import numpy as np

# Definimos parámetros
beta_0 = -3
beta_1 = 0.8
omega = np.diag([4, 9, 16, 25, 36])
N = 5  # Observaciones por muestra
M = 5000  # Cantidad de muestras

# Generación de datos
np.random.seed(3649)
x = np.random.uniform(1, 50, size=(M, N))  # x ~ U[1, 50]
chol_omega = np.linalg.cholesky(omega)     # P = Cholesky de omega
u = chol_omega @ np.random.randn(N, M)    # u ~ N(0, omega)
u = u.T  # Transpuesta para mantener dimensiones MxN
y = beta_0 + beta_1 * x + u               # y_i = beta_0 + beta_1 * x_i + u_i

# Transformación para MCC
P_inv = np.linalg.inv(chol_omega)         # Inversa de P
y_star = y @ P_inv.T                      # y* = P^-1 * y
x_star = x @ P_inv.T                      # X* = P^-1 * X
x_star = np.hstack([np.ones((M, 1)), x_star])  # Agregar constante a X*

# Estimación MCC
beta_mcc = np.linalg.inv(x_star.T @ x_star) @ x_star.T @ y_star

# Resultados
print("Estimación por MCC (beta_0, beta_1):")
print(beta_mcc.mean(axis=0))  # Promedio de betas estimadas en todas las muestras

Estimación por MCC (beta_0, beta_1):
[-0.10411783 -0.04700603  0.00813422  0.03736534  0.06757768]


In [13]:
beta_mcc[0]

array([-1.41230844, -1.08667462, -0.74686156, -0.56526369, -0.38246838])

In [8]:
x[0]

array([44.69194777, 33.9455005 , 37.36404406, 37.25104984, 34.52517416])

In [11]:
np.hstack([np.ones((M, 1)), x]).shape

(5000, 6)

In [21]:
display(omega, chol_omega)

array([[ 4,  0,  0,  0,  0],
       [ 0,  9,  0,  0,  0],
       [ 0,  0, 16,  0,  0],
       [ 0,  0,  0, 25,  0],
       [ 0,  0,  0,  0, 36]])

array([[2., 0., 0., 0., 0.],
       [0., 3., 0., 0., 0.],
       [0., 0., 4., 0., 0.],
       [0., 0., 0., 5., 0.],
       [0., 0., 0., 0., 6.]])

In [17]:
import numpy as np
import pandas as pd

# Configuración inicial
np.random.seed(3649)

# Parámetros iniciales
beta_0_true = -3
beta_1_true = 0.8
n_observations = 5  # Observaciones por muestra
n_samples = 5000  # Número de muestras

# Matriz omega (varianzas)
omega = np.diag([4, 9, 16, 25, 36])

# Descomposición de Cholesky de omega
P = np.linalg.cholesky(omega)

# Inversa de P
P_inv = np.linalg.inv(P)

# Contenedores para almacenar muestras y estimadores
samples = []
results = []

# Simulación de M muestras
for sample_id in range(n_samples):
    # 1. Generar datos
    x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
    u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
    y = beta_0_true + beta_1_true * x + u  # Generar y

    # Almacenar muestra
    samples.append(pd.DataFrame({'sample': i, 'x': x, 'y': y, 'u': u}))

    # 2. Transformar para MCG
    y_tilde = P_inv @ y
    X = np.column_stack((np.ones(n_observations), x))  # Matriz de diseño original
    X_tilde = P_inv @ X

    # 3. Ajustar modelo transformado con MCO
    beta_hat = np.linalg.inv(X_tilde.T @ X_tilde) @ (X_tilde.T @ y_tilde)

    # Almacenar resultados
    results.append([i, beta_0_hat, beta_1_hat, se_beta1_hat])

# Combinar todas las muestras en un único DataFrame
all_samples = pd.concat(samples).reset_index(drop=True)

# Crear DataFrame con los resultados de FGLS
results_df = pd.DataFrame(results, columns=[
    'sample', 'beta_0_hat', 'beta_1_hat', 'se_beta1_hat'
])

# Mostrar ejemplos de los DataFrames generados
print("Ejemplo de all_samples:")
print(all_samples.head())
print("\nEjemplo de results_df:")
print(results_df.head())

NameError: name 'beta_0_hat' is not defined

In [19]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Configuración inicial
np.random.seed(3649)

# Parámetros iniciales
beta_0_true = -3
beta_1_true = 0.8
n_observations = 5  # Observaciones por muestra
n_samples = 5000  # Número de muestras

# Matriz omega (varianzas)
omega = np.diag([4, 9, 16, 25, 36])

# Descomposición de Cholesky de omega
P = np.linalg.cholesky(omega)

# Inversa de P
P_inv = np.linalg.inv(P)

# Contenedores para almacenar muestras y estimadores
samples = []
results = []

# Simulación de M muestras
for i in range(n_samples):
    # 1. Generar datos
    x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
    u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
    y = beta_0_true + beta_1_true * x + u  # Generar y

    # Almacenar muestra
    samples.append(pd.DataFrame({'sample': i, 'x': x, 'y': y, 'u': u}))

    # 2. Transformar para MCG
    y_tilde = P_inv @ y
    X = sm.add_constant(x)  # Agregar columna de unos automáticamente
    X_tilde = P_inv @ X

    # 3. Ajustar modelo transformado con statsmodels
    model = sm.OLS(y_tilde, X_tilde)
    results_model = model.fit()

    # Extraer coeficientes y error estándar
    beta_0_hat, beta_1_hat = results_model.params
    se_beta1_hat = results_model.bse[1]  # Error estándar del coeficiente beta_1

    # Almacenar resultados
    results.append([i, beta_0_hat, beta_1_hat, se_beta1_hat])

# Combinar todas las muestras en un único DataFrame
all_samples = pd.concat(samples).reset_index(drop=True)

# Crear DataFrame con los resultados de FGLS
results_df = pd.DataFrame(results, columns=[
    'sample', 'beta_0_hat', 'beta_1_hat', 'se_beta1_hat'
])

# Mostrar ejemplos de los DataFrames generados
print("Ejemplo de all_samples:")
print(all_samples.head())
print("\nEjemplo de results_df:")
print(results_df.head())

Ejemplo de all_samples:
   sample          x          y         u
0       0  44.691948  33.147926  0.394367
1       0  33.945501  21.417052 -2.739349
2       0  37.364044  21.773406 -5.117829
3       0  37.251050  28.303198  1.502358
4       0  34.525174  30.658162  6.038023

Ejemplo de results_df:
   sample  beta_0_hat  beta_1_hat  se_beta1_hat
0       0  -11.213024    0.989465      0.301298
1       1   -3.104349    0.893636      0.119634
2       2    1.983310    0.630313      0.192682
3       3   -1.290987    0.670113      0.142292
4       4   -0.085546    0.737440      0.122292


In [ ]:
from scipy.stats import t

# Parámetros iniciales
beta_0 = -3
beta_1_true = 0.8
omega = np.diag([4, 9, 16, 25, 36])  # Matriz de varianzas
n_samples = 5000
n_observations = 5
alpha_levels = [0.01, 0.05]  # Niveles de significancia

# Función para estimar por FGLS
def fgls_estimation(x, y, omega):
    W = np.linalg.inv(omega)  # Matriz de pesos inversos
    X = np.column_stack((np.ones(len(x)), x))  # Matriz de diseño
    beta_hat = np.linalg.inv(X.T @ W @ X) @ (X.T @ W @ y)  # Estimación FGLS
    var_beta = np.linalg.inv(X.T @ W @ X)  # Varianza de los estimadores
    return beta_hat, np.sqrt(np.diag(var_beta))  # Estimadores y errores estándar

# Simulaciones
results = []
np.random.seed(3649)

for _ in range(n_samples):
    x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
    u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
    y = beta_0 + beta_1_true * x + u  # Generar y
    beta_hat, se_beta = fgls_estimation(x, y, omega)  # Estimación FGLS
    t_stat = (beta_hat[1] - beta_1_true) / se_beta[1]  # Estadístico t para beta_1
    p_value = 2 * (1 - t.cdf(abs(t_stat), df=n_observations - 2))  # Valor p
    results.append([beta_hat[0], beta_hat[1], se_beta[0], se_beta[1], t_stat, p_value])

# Convertir resultados a DataFrame
results_df = pd.DataFrame(results, columns=['beta_0', 'beta_1', 'se_beta_0', 'se_beta_1', 't_stat', 'p_value'])

# Tamaño del test para diferentes niveles de significancia
size_test = {
    alpha: (results_df['p_value'] < alpha).mean() for alpha in alpha_levels
}

# Poder del test para diferentes valores alternativos de beta_1
power_results = {}
for beta_1_alt in [0, 0.4]:
    power = []
    for _ in range(n_samples):
        x = np.random.uniform(1, 50, n_observations)
        u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)
        y = beta_0 + beta_1_alt * x + u
        beta_hat, se_beta = fgls_estimation(x, y, omega)
        t_stat = (beta_hat[1] - beta_1_true) / se_beta[1]
        p_value = 2 * (1 - t.cdf(abs(t_stat), df=n_observations - 2))
        power.append(p_value < 0.05)
    power_results[beta_1_alt] = np.mean(power)

# Estadísticas descriptivas
descriptive_stats = results_df[['beta_0', 'beta_1']].agg(['mean', 'median', 'std'])

# Resultados finales
print("Tamaño del test:", size_test)
print("Poder del test:", power_results)
print("Estadísticas descriptivas:\n", descriptive_stats)

Tamaño del test: {0.01: 0.0, 0.05: 0.0012}
Poder del test: {0: 0.9014, 0.4: 0.452}
Estadísticas descriptivas:
           beta_0    beta_1
mean   -2.958325  0.798089
median -2.890889  0.796406
std     4.649156  0.162026


In [ ]:
import statsmodels.api as sm
from scipy.stats import t

# Parámetros iniciales
beta_0 = -3
beta_1_true = 0.8
omega = np.diag([4, 9, 16, 25, 36])  # Matriz de varianzas
n_samples = 5000
n_observations = 5
alpha_levels = [0.01, 0.05]  # Niveles de significancia

# Función para estimar por FGLS usando statsmodels
def fgls_estimation(x, y, omega):
    X = sm.add_constant(x)  # Matriz de diseño con intercepto
    model = sm.GLS(y, X, sigma=omega)  # Estimación FGLS con sigma = omega
    results = model.fit()  # Ajustar el modelo
    return results

# Simulaciones
results = []
np.random.seed(3649)

for _ in range(n_samples):
    x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
    u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
    y = beta_0 + beta_1_true * x + u  # Generar y
    result = fgls_estimation(x, y, omega)  # Estimación FGLS
    t_stat = result.tvalues[1]  # Estadístico t para beta_1
    p_value = result.pvalues[1]  # Valor p para beta_1
    results.append([result.params[0], result.params[1], result.bse[0], result.bse[1], t_stat, p_value])

# Convertir resultados a DataFrame
results_df = pd.DataFrame(results, columns=['beta_0', 'beta_1', 'se_beta_0', 'se_beta_1', 't_stat', 'p_value'])

# Tamaño del test para diferentes niveles de significancia
size_test = {
    alpha: (results_df['p_value'] < alpha).mean() for alpha in alpha_levels
}

# Poder del test para diferentes valores alternativos de beta_1
power_results = {}
for beta_1_alt in [0, 0.4]:
    power = []
    for _ in range(n_samples):
        x = np.random.uniform(1, 50, n_observations)
        u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)
        y = beta_0 + beta_1_alt * x + u
        result = fgls_estimation(x, y, omega)
        t_stat = result.tvalues[1]
        p_value = result.pvalues[1]
        power.append(p_value < 0.05)
    power_results[beta_1_alt] = np.mean(power)

# Estadísticas descriptivas
descriptive_stats = results_df[['beta_0', 'beta_1']].agg(['mean', 'median', 'std'])

# Resultados finales
print("Tamaño del test:", size_test)
print("Poder del test:", power_results)
print("Estadísticas descriptivas:\n", descriptive_stats)

Tamaño del test: {0.01: 0.6018, 0.05: 0.8908}
Poder del test: {0: 0.0532, 0.4: 0.5276}
Estadísticas descriptivas:
           beta_0    beta_1
mean   -2.958325  0.798089
median -2.890889  0.796406
std     4.649156  0.162026


In [10]:
import numpy as np
import statsmodels.api as sm
import pandas as pd

# Parámetros
beta_0 = -3
beta_1_true = 0.8
omega = np.diag([4, 9, 16, 25, 36])  # Matriz de varianzas
n_observations = 5  # Pequeña muestra
n_samples = 5000  # Número de simulaciones

# Generar datos
np.random.seed(42)

def generate_data():
    x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
    u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
    y = beta_0 + beta_1_true * x + u  # Generar y
    return x, y

# Método 1: Estimación con FGLS usando statsmodels
def fgls_statsmodels(x, y):
    X = sm.add_constant(x)  # Matriz de diseño con intercepto
    model = sm.GLS(y, X, sigma=omega)  # Estimación FGLS con sigma = omega
    results = model.fit()  # Ajustar el modelo
    return results

# Método 2: Estimación manual FGLS
def fgls_manual(x, y, omega):
    X = np.column_stack([np.ones(n_observations), x])  # Matriz de diseño con intercepto
    # Calcular la inversa de Omega
    omega_inv = np.linalg.inv(omega)
    # Estimar Beta usando la fórmula FGLS
    beta_hat = np.linalg.inv(X.T @ omega_inv @ X) @ (X.T @ omega_inv @ y)
    # Estimación de errores estándar (desviación estándar)
    residuals = y - X @ beta_hat
    sigma_hat = (residuals.T @ omega_inv @ residuals) / (n_observations - 2)  # Varianza de los errores
    cov_beta_hat = np.linalg.inv(X.T @ omega_inv @ X) * sigma_hat  # Varianza de los coeficientes
    se_beta_hat = np.sqrt(np.diag(cov_beta_hat))  # Desviación estándar de los coeficientes
    return beta_hat, se_beta_hat

# Comparación de ambos métodos
results_fgls_sm = []
results_fgls_manual = []

for _ in range(n_samples):
    x, y = generate_data()
    
    # Método 1: FGLS con statsmodels
    result_sm = fgls_statsmodels(x, y)
    results_fgls_sm.append([result_sm.params[0], result_sm.params[1], result_sm.bse[0], result_sm.bse[1]])
    
    # Método 2: Estimación manual
    beta_hat_manual, se_beta_hat_manual = fgls_manual(x, y, omega)
    results_fgls_manual.append([beta_hat_manual[0], beta_hat_manual[1], se_beta_hat_manual[0], se_beta_hat_manual[1]])

# Convertir resultados a DataFrame para comparar
results_fgls_sm_df = pd.DataFrame(results_fgls_sm, columns=['beta_0_sm', 'beta_1_sm', 'se_beta_0_sm', 'se_beta_1_sm'])
results_fgls_manual_df = pd.DataFrame(results_fgls_manual, columns=['beta_0_manual', 'beta_1_manual', 'se_beta_0_manual', 'se_beta_1_manual'])

# Comparar medias, medianas y desviaciones estándar
comparison = pd.concat([
    results_fgls_sm_df[['beta_0_sm', 'beta_1_sm', 'se_beta_0_sm', 'se_beta_1_sm']].agg(['mean', 'median', 'std']),
    results_fgls_manual_df[['beta_0_manual', 'beta_1_manual', 'se_beta_0_manual', 'se_beta_1_manual']].agg(['mean', 'median', 'std'])
], axis=1)

comparison

,beta_0_sm,beta_1_sm,se_beta_0_sm,se_beta_1_sm,beta_0_manual,beta_1_manual,se_beta_0_manual,se_beta_1_manual
mean,-3.124395,0.803626,3.777375,0.136698,-3.124395,0.803626,3.777375,0.136698
median,-3.050941,0.804804,3.135215,0.118106,-3.050941,0.804804,3.135215,0.118106
std,4.543793,0.162838,2.673090,0.088475,4.543793,0.162838,2.673090,0.088475


In [12]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
from scipy.stats import t

# Parámetros iniciales
beta_0 = -3
beta_1_true = 0.8
omega = np.diag([4, 9, 16, 25, 36])  # Matriz de varianzas
n_samples = 5000
n_observations = 5
alpha_levels = [0.01, 0.05]  # Niveles de significancia

# Función para estimar por FGLS usando statsmodels con errores robustos
def fgls_estimation(x, y, omega):
    X = sm.add_constant(x)  # Matriz de diseño con intercepto
    model = sm.GLS(y, X, sigma=omega)  # Estimación FGLS con sigma = omega
    results = model.fit()  # Ajustar el modelo
    robust_results = results.get_robustcov_results(cov_type='HC3')  # Errores estándar robustos (HC3)
    return robust_results

# Simulaciones
results = []
np.random.seed(3649)

for _ in range(n_samples):
    x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
    u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
    y = beta_0 + beta_1_true * x + u  # Generar y
    robust_results = fgls_estimation(x, y, omega)  # Estimación FGLS con errores robustos
    t_stat = robust_results.tvalues[1]  # Estadístico t para beta_1
    p_value = robust_results.pvalues[1]  # Valor p para beta_1
    results.append([robust_results.params[0], robust_results.params[1], robust_results.bse[0], robust_results.bse[1], t_stat, p_value])

# Convertir resultados a DataFrame
results_df = pd.DataFrame(results, columns=['beta_0', 'beta_1', 'se_beta_0', 'se_beta_1', 't_stat', 'p_value'])

# Tamaño del test para diferentes niveles de significancia
size_test = {
    alpha: (results_df['p_value'] < alpha).mean() for alpha in alpha_levels
}

# Poder del test para diferentes valores alternativos de beta_1
power_results = {}
for beta_1_alt in [0, 0.4]:
    power = []
    for _ in range(n_samples):
        x = np.random.uniform(1, 50, n_observations)
        u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)
        y = beta_0 + beta_1_alt * x + u
        robust_results = fgls_estimation(x, y, omega)
        t_stat = robust_results.tvalues[1]
        p_value = robust_results.pvalues[1]
        power.append(p_value < 0.05)
    power_results[beta_1_alt] = np.mean(power)

# Estadísticas descriptivas
descriptive_stats = results_df[['beta_0', 'beta_1']].agg(['mean', 'median', 'std'])

# Resultados finales
print("Tamaño del test:", size_test)
print("Poder del test:", power_results)
print("Estadísticas descriptivas:\n", descriptive_stats)

Tamaño del test: {0.01: 0.298, 0.05: 0.6176}
Poder del test: {0: 0.0282, 0.4: 0.2734}
Estadísticas descriptivas:
           beta_0    beta_1
mean   -2.958325  0.798089
median -2.890889  0.796406
std     4.649156  0.162026


In [13]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

# Parámetros del modelo
beta_0 = -3
beta_1_true = 0.8
omega = np.diag([4, 9, 16, 25, 36])  # Matriz de varianzas (heterocedasticidad)
n_samples = 5000
n_observations = 5
alpha_levels = [0.01, 0.05]  # Niveles de significancia

# Función para realizar la estimación FGLS con errores robustos de White
def fgls_white_estimation(x, y, omega):
    # Paso 1: Estimación OLS inicial
    X = sm.add_constant(x)  # Matriz de diseño con intercepto
    ols_model = sm.OLS(y, X)  # Estimación OLS
    ols_results = ols_model.fit()  # Ajuste de OLS
    
    # Paso 2: Estimación de la matriz de varianzas-covarianzas de los errores
    residuals = ols_results.resid  # Residuos de la estimación OLS
    # Usamos la matriz de varianzas-covarianzas robusta (White)
    robust_cov = ols_results.get_robustcov_results(cov_type='HC3').cov_params()  # Matriz robusta de varianzas-covarianzas
    
    # Paso 3: Estimación FGLS con la matriz robusta
    W = np.linalg.inv(robust_cov)  # Matriz de pesos (inversa de la matriz de varianzas-covarianzas)
    fgls_model = sm.GLS(y, X, sigma=W)  # Estimación FGLS usando la matriz robusta
    fgls_results = fgls_model.fit()  # Ajuste FGLS
    
    return fgls_results

# Simulación de los datos
results = []
np.random.seed(3649)

for _ in range(n_samples):
    x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
    u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
    y = beta_0 + beta_1_true * x + u  # Generar y
    fgls_result = fgls_white_estimation(x, y, omega)  # Estimación FGLS White
    
    # Obtener estadísticos de interés
    t_stat = fgls_result.tvalues[1]  # Estadístico t para beta_1
    p_value = fgls_result.pvalues[1]  # Valor p para beta_1
    results.append([fgls_result.params[0], fgls_result.params[1], fgls_result.bse[0], fgls_result.bse[1], t_stat, p_value])

# Convertir resultados a DataFrame
results_df = pd.DataFrame(results, columns=['beta_0', 'beta_1', 'se_beta_0', 'se_beta_1', 't_stat', 'p_value'])

# Tamaño del test para diferentes niveles de significancia
size_test = {
    alpha: (results_df['p_value'] < alpha).mean() for alpha in alpha_levels
}

# Poder del test para diferentes valores alternativos de beta_1
power_results = {}
for beta_1_alt in [0, 0.4]:
    power = []
    for _ in range(n_samples):
        x = np.random.uniform(1, 50, n_observations)
        u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)
        y = beta_0 + beta_1_alt * x + u
        fgls_result = fgls_white_estimation(x, y, omega)
        p_value = fgls_result.pvalues[1]
        power.append(p_value < 0.05)
    power_results[beta_1_alt] = np.mean(power)

# Estadísticas descriptivas
descriptive_stats = results_df[['beta_0', 'beta_1']].agg(['mean', 'median', 'std'])

# Resultados finales
print("Tamaño del test:", size_test)
print("Poder del test:", power_results)
print("Estadísticas descriptivas:\n", descriptive_stats)

ValueError: Sigma must be a scalar, 1d of length 5 or a 2d array of shape 5 x 5